In [1]:
import pandas as pd

In [3]:
# Path to the txt file
file_path = './SOBRADINHO_DIARIA_1931_2016.txt'

# Reading file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Creating a DataFrame
df = pd.DataFrame(lines, columns=['Flow'])

# Strip newline characters from the end of each line
df['Flow'] = df['Flow'].str.strip()

print(df)

       Flow
0      5512
1      5543
2      5607
3      5639
4      5703
...     ...
31407  2871
31408  3048
31409  3163
31410  3160
31411  3023

[31412 rows x 1 columns]
